<a href="https://colab.research.google.com/github/aliarman1/DataWarehouse/blob/main/DM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
import warnings

In [4]:
df = pd.read_csv('ID_56_and_45_Datamining_Dataset.csv')
df.head()

,content,class,Unnamed: 2
0,"In this study, aim to generate DTM that...",0,0 = Original \n1 = AI
1,"In this study, our objective is to create a Di...",1,NaN
2,"In this study, our objective is to create a Di...",1,NaN
3,Kaguya is a large satellite was launched from ...,0,NaN
4,"Kaguya, a substantial satellite launched from ...",1,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   content     770 non-null    object
 1   class       770 non-null    int64 
 2   Unnamed: 2  1 non-null      object
dtypes: int64(1), object(2)
memory usage: 18.2+ KB


In [6]:
#Removing Special Charcter
df['clean_content'] = df['content'].str.replace("[^a-zA-Z0-9#]", " ")
df.head()

<ipython-input-6-f0a55e3e532f>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_content'] = df['content'].str.replace("[^a-zA-Z0-9#]", " ")


,content,class,Unnamed: 2,clean_content
0,"In this study, aim to generate DTM that...",0,0 = Original \n1 = AI,In this study aim to generate DTM that...
1,"In this study, our objective is to create a Di...",1,NaN,In this study our objective is to create a Di...
2,"In this study, our objective is to create a Di...",1,NaN,In this study our objective is to create a Di...
3,Kaguya is a large satellite was launched from ...,0,NaN,Kaguya is a large satellite was launched from ...
4,"Kaguya, a substantial satellite launched from ...",1,NaN,Kaguya a substantial satellite launched from ...


In [7]:
## remove short words
df['clean_content'] = df['clean_content'].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
df.head()

,content,class,Unnamed: 2,clean_content
0,"In this study, aim to generate DTM that...",0,0 = Original \n1 = AI,this study generate that express detail terrai...
1,"In this study, our objective is to create a Di...",1,NaN,this study objective create Digital Terrain Mo...
2,"In this study, our objective is to create a Di...",1,NaN,this study objective create Digital Terrain Mo...
3,Kaguya is a large satellite was launched from ...,0,NaN,Kaguya large satellite launched from Japan 200...
4,"Kaguya, a substantial satellite launched from ...",1,NaN,Kaguya substantial satellite launched from Jap...


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [9]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['clean_content'])
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df['clean_content'])
padded_sequences = pad_sequences(sequences, maxlen=10, padding='post', truncating='post')


In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, df['class'], random_state=42, test_size=0.25)

In [11]:
model = keras.Sequential([
    keras.layers.Embedding(1000, 16, input_length=10),
    keras.layers.Flatten(),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

X = np.array(x_train)
y = np.array(y_train)
model.fit(X, y, epochs=10, verbose=2)

Epoch 1/10
19/19 - 1s - loss: 0.6883 - accuracy: 0.5893 - 932ms/epoch - 49ms/step
Epoch 2/10
19/19 - 0s - loss: 0.6680 - accuracy: 0.7244 - 35ms/epoch - 2ms/step
Epoch 3/10
19/19 - 0s - loss: 0.6451 - accuracy: 0.7227 - 36ms/epoch - 2ms/step
Epoch 4/10
19/19 - 0s - loss: 0.6152 - accuracy: 0.7262 - 35ms/epoch - 2ms/step
Epoch 5/10
19/19 - 0s - loss: 0.5792 - accuracy: 0.7366 - 34ms/epoch - 2ms/step
Epoch 6/10
19/19 - 0s - loss: 0.5415 - accuracy: 0.7626 - 39ms/epoch - 2ms/step
Epoch 7/10
19/19 - 0s - loss: 0.4997 - accuracy: 0.8059 - 45ms/epoch - 2ms/step
Epoch 8/10
19/19 - 0s - loss: 0.4505 - accuracy: 0.8423 - 42ms/epoch - 2ms/step
Epoch 9/10
19/19 - 0s - loss: 0.4001 - accuracy: 0.8943 - 32ms/epoch - 2ms/step
Epoch 10/10
19/19 - 0s - loss: 0.3469 - accuracy: 0.9237 - 34ms/epoch - 2ms/step


In [12]:
c = 2
predictions = model.predict(x_test[c:c+1])
predictions
y_test[c:c+1]
q = "AI" if int(y_test[c:c+1]) > 0.5 else "Normal"
p = "AI" if predictions > 0.5 else "Normal"
print("Predicted Answer: "+ p)
print("Original Answer: "+ q)

1/1 [==============================] - 0s 95ms/step
Predicted Answer: Normal
Original Answer: Normal


In [13]:
c = 5
predictions = model.predict(x_test[c:c+1])
predictions
y_test[c:c+1]
q = "AI" if int(y_test[c:c+1]) > 0.5 else "Normal"
p = "AI" if predictions > 0.5 else "Normal"
print("Predicted Answer: "+ p)
print("Original Answer: "+ q)

1/1 [==============================] - 0s 24ms/step
Predicted Answer: AI
Original Answer: AI


In [14]:
new_texts = ["In this study, our objective is to create a Digital Terrain Model", "Measuring center of Ball Ball  has  11  points  of  reflection  sheets.  In  their points,  the  defect  frames  were  interpolated  by  the Lagrange interpolation using Excel VBA."]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=10, padding='post', truncating='post')
predictions = model.predict(new_padded_sequences)
predictions
for text, prediction in zip(new_texts, predictions):
    p = "AI" if prediction > 0.5 else "Normal"
    print(f'Text: {text} | Prediction: {p} | Confidence: {prediction[0]:.2f}')

1/1 [==============================] - 0s 19ms/step
Text: In this study, our objective is to create a Digital Terrain Model | Prediction: AI | Confidence: 0.73
Text: Measuring center of Ball Ball  has  11  points  of  reflection  sheets.  In  their points,  the  defect  frames  were  interpolated  by  the Lagrange interpolation using Excel VBA. | Prediction: AI | Confidence: 0.59
